In [11]:
import numpy as np
import pandas as pd
import scipy as sci
from ase import Atoms
from ase.visualize import view
from ase.io import write
from ase.io import read
import os
import subprocess
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import rdMolTransforms

In [12]:
#read file containing all of the paths of the for the structures to analyze
df = pd.read_csv('thermo_out_surf.csv')

In [13]:
ev_kJ = 96.4869
h_ev = 4.135668E-15 #planks constant in ev*s
k_ev = 0.0000861733 #boltzmann in ev/K
R_ig = 8.3144 #kJ/mol/K
T = 533

#initial vector to build S H and ZPEs
S_vec = np.array([])
H_vec = np.array([])
ZPE_vec = np.array([])
H_rot_vec = np.array([])
H_trans_vec = np.array([])
S_rot_ig_vec = np.array([])
S_rot_vec = np.array([])
S_trans_vec = np.array([])
f_ims = np.array([])

In [14]:

lp = df['path_g'][0]
f = open(lp+'/CONTCAR','r')
data = list([])
for lines in f:
    data.append(lines)
    
atoms = data[6].split(' ')
ele=''
atoms=[i for i in atoms if i!=ele]
atoms[-1] = atoms[-1][0:-1]
n_HC = list([])
[n_HC.append(int(x)) for x in atoms]
n_atoms = sum(n_HC[1:])

In [15]:
for i in range(df.shape[0]):
    [N_A,k_B,h,kJmol_Ha] = [6.0221408e23,1.380649e-23,6.626070E-34,2625.5] #[ -- , m2 kg s-2 K-1, K,  # m2 kg / s]
    R_ig = 8.3144
    R = k_B*N_A 
    
    lp = df['path_f'][i]
    print(lp)
    #get number of atoms at DOF
    f = open(df['path_g'][i]+'/CONTCAR','r')
    data = list([])
    
    for lines in f:
        data.append(lines)
    atoms = data[6].split(' ')
    ele=''
    atoms=[i for i in atoms if i!=ele]
    atoms[-1] = atoms[-1][0:-1]
    n_HC = list([])
    [n_HC.append(int(x)) for x in atoms]
    n_atoms = sum(n_HC[1:])
    
    DoF = n_atoms*3
    m = df['mass'][i]*1.67377E-27
    
    
    #moments
    raw_mol = Chem.MolFromXYZFile(df['path_g'][i]+'/out.xyz')
    mol = Chem.Mol(raw_mol)
    conf = mol.GetConformer()
    moments_of_inertia = rdMolTransforms.ComputePrincipalAxesAndMoments(conf)
    inertia_values = moments_of_inertia[1]*1.67377E-27/1e10/1e10
    print(inertia_values)
    
    try:
        df_freq = pd.read_csv(lp+'/Freqs.txt',header=None,delim_whitespace=True)

    except ValueError:
        df_freq = np.array([])
    
    if np.size(df_freq)!= 0:
        df_freq = pd.read_csv(lp+'/Freqs.txt',header=None,delim_whitespace=True)
        vibs = df_freq[7]
        vibs = pd.to_numeric(vibs,errors='coerce')

        n_low_freq = sum(vibs<100)#+(DoF-np.size(vibs))
        
        #convert to hz
        v_hz = vibs.to_numpy()*29979.2458*1e6#cm-1 -> Mhz -> hz
        S_vibs =R_ig/1000*((h_ev*v_hz/(k_ev*T*(np.exp(h_ev*v_hz/k_ev/T)-1))-np.log(1-np.exp(-h_ev*v_hz/k_ev/T))))
        ZPE = 1/2*h_ev*v_hz
        H_vib = h_ev*v_hz*np.exp(-h_ev*v_hz/k_ev/T)/(1-np.exp(-h_ev*v_hz/k_ev/T))
        
        #populate table with S H and ZPE values:
        #for minima keep all modes
        S_vec = np.append(S_vec,np.sum(np.sum(S_vibs[0:len(vibs)-n_low_freq]))) 
        H_vec = np.append(H_vec,np.sum(np.sum(H_vib[0:len(vibs)-n_low_freq]))) 
        ZPE_vec = np.append(ZPE_vec,np.sum(np.sum(ZPE))) 

    else:
        S_vec = np.append(S_vec,0) 
        H_vec = np.append(H_vec,0) 
        ZPE_vec = np.append(ZPE_vec,0) 
        
    #calculate rotational enthalpy and entropy using moments of inertia
    sigma = 1
    A = h/8/np.pi/np.pi/inertia_values[0]
    B = h/8/np.pi/np.pi/inertia_values[1]
    C = h/8/np.pi/np.pi/inertia_values[2]
    H_rot_ig = 3/2*R_ig*T/1000 # kJ/mol    
    S_rot_ig = R_ig/1000*(3/2*np.log(k_B*T/h)-1/2*np.log(A*B*C/np.pi)-np.log(sigma)+3/2)
    H_trans_ig = 5/2*R_ig*T/1000
    S_trans_ig = R_ig/1000*(3/2*np.log(2*np.pi*m/h/h)+5/2*np.log(k_B*T)-np.log(1E5)+5/2) # J/mol/K      
    
    
    #populate the table
    H_rot = 0.7*(n_low_freq+(DoF-np.size(vibs)))/6*(H_trans_ig+H_rot_ig) # kJ/mol 
    H_trans = 0 # kJ/mol
    S_rot = 0.7*(n_low_freq+(DoF-np.size(vibs)))/6*(S_trans_ig+S_rot_ig) # J/mol/K
    S_trans = 0 # J/mol/K    

    H_rot_vec = np.append(H_rot_vec,H_rot)
    H_trans_vec = np.append(H_trans_vec,H_trans)
    S_rot_vec = np.append(S_rot_vec,S_rot)
    S_trans_vec = np.append(S_trans_vec,S_trans)
        
df.insert(2,'S_vib',S_vec) #kJ/mol
df.insert(2,'H_vib',H_vec*ev_kJ) #kJ/mol 
df.insert(2,'ZPE',ZPE_vec*ev_kJ) #kJ/mol

df.insert(2,'H_rot',H_rot_vec) #kcal/mol -> kJ/mol (occured at import)
df.insert(2,'H_trans',H_trans_vec) #kcal/mol -> kJ/mol
df.insert(2,'S_rot',S_rot_vec) #kcal/mol -> J/mol/K
df.insert(2,'S_trans',S_trans_vec) #kcal/mol -> J/mol/K

/Users/arifischer/Documents/misc/MS2/DFT_calculations/compiling_calculations/methylenebenzene/freq
[1.03124416e-46 2.26351002e-46 3.29328709e-46]
0     3082.265163
1     3079.022785
2     3068.095277
3     3057.544835
4     3045.946057
5     3042.529933
6     2987.466305
7     1468.370443
8     1448.158687
9     1425.777851
10    1417.591193
11    1379.908752
12    1330.891718
13    1294.609802
14    1215.860378
15    1158.350999
16    1149.991468
17    1049.340054
18     980.820647
19     975.043567
20     966.976767
21     932.643845
22     914.687433
23     906.504723
24     858.735995
25     832.684610
26     806.155447
27     787.975826
28     749.282291
29     637.079498
30     581.079167
31     512.119177
32     484.066625
33     414.056328
34     387.457878
35     370.569868
36     254.255043
37     220.170252
38     165.135039
39     126.710650
40      96.089592
41      31.023686
Name: 7, dtype: float64
/Users/arifischer/Documents/misc/MS2/DFT_calculations/compiling_calculatio

In [16]:
#free energies:

#G = H - T S
#H + E + ZPE + H_vib
Hs = df['E_ev'].to_numpy()*ev_kJ + df['ZPE'].to_numpy() + df['H_vib'].to_numpy()
Ss = df['S_vib'].to_numpy()+df['S_rot'].to_numpy()+df['S_trans'].to_numpy()
Gs = Hs - T*Ss
Gs

array([-22392.44044345, -22726.26686274, -23046.84327214, -23420.43711442,
       -23376.9735371 , -23731.60024391, -24063.22309249, -24079.05539706,
       -24415.97458942, -24435.34872119])

In [17]:
#build dataframe to export thermodynamics inforamtion
df.insert(1,'H_533',Hs) #kcal/mol -> kJ/mol (occured at import)
df.insert(1,'S_533',Ss) #kcal/mol -> kJ/mol (occured at import)
df.insert(1,'G_533',Gs) #kcal/mol -> kJ/mol (occured at import)

In [18]:
#df.to_csv('ads_thermo_533.csv')